In [3]:
import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_val, y_val) = keras.datasets.fashion_mnist.load_data()

In [4]:
def preprocess(x,y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)

    return x,y

def create_dataset(xs, ys, n_classes=10):
    ys = tf.one_hot(ys, depth=n_classes)

    return tf.data.Dataset.from_tensor_slices((xs,ys)) \
        .map(preprocess) \
        .shuffle(len(ys)) \
        .batch(128)

In [5]:
train_ds = create_dataset(x_train,y_train)
val_ds = create_dataset(x_val, y_val)


In [11]:
model = keras.Sequential([
    keras.layers.Reshape(
        target_shape=(28 * 28,), input_shape=(28,28)
    ),
    keras.layers.Dense(
        units=256, activation='relu'
    ),
    keras.layers.Dense(
        units=192, activation='relu'
    ),
    keras.layers.Dense(
        units=128, activation='relu'
    ),
    keras.layers.Dense(
        units=10, activation='softmax'
    )
])

In [12]:
model.compile(
    optimizer='adam',
    loss=tf.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history = model.fit(
    train_ds.repeat(),
    epochs=10,
    steps_per_epoch=500,
    validation_data=val_ds.repeat(),
    validation_steps=2
)

Train for 500 steps, validate for 2 steps
Epoch 1/10
500/500 [==============================] - 5s 10ms/step - loss: 1.7504 - accuracy: 0.7164 - val_loss: 1.7095 - val_accuracy: 0.7578
Epoch 2/10
500/500 [==============================] - 3s 6ms/step - loss: 1.7196 - accuracy: 0.7417 - val_loss: 1.7136 - val_accuracy: 0.7461
Epoch 3/10
500/500 [==============================] - 3s 6ms/step - loss: 1.6982 - accuracy: 0.7626 - val_loss: 1.6558 - val_accuracy: 0.8047
Epoch 4/10
500/500 [==============================] - 3s 6ms/step - loss: 1.6732 - accuracy: 0.7876 - val_loss: 1.7067 - val_accuracy: 0.7539
Epoch 5/10
500/500 [==============================] - 3s 6ms/step - loss: 1.6651 - accuracy: 0.7956 - val_loss: 1.6878 - val_accuracy: 0.7734
Epoch 6/10
500/500 [==============================] - 3s 6ms/step - loss: 1.6588 - accuracy: 0.8021 - val_loss: 1.6958 - val_accuracy: 0.7617
Epoch 7/10
500/500 [==============================] - 3s 6ms/step - loss: 1.6582 - accuracy: 0.8026 - val

In [23]:
predictions = model.predict(val_ds)
best_predictions = [i.argmax() for i in predictions]
correct = 0
for i in range(len(best_predictions)):
    if best_predictions[i] == y_val[i]:
        correct += 1
print(correct/len(y_val)*100)

10.4


In [24]:
model.save('fashion_mnist(10.4).h5')